In [129]:
import json
import re
import os
import ast
import csv
import pandas as pd

In [130]:
def decode_utf8(item):
    if isinstance(item, str):  # If item is a string
        try:
            return item.encode('latin1').decode('utf-8')
        except:
            return item  # Return original if decoding fails
    elif isinstance(item, list):  # If item is a list
        return [decode_utf8(x) for x in item]
    elif isinstance(item, dict):  # If item is a dictionary
        return {key: decode_utf8(value) for key, value in item.items()}
    else:  # If item is neither string, list nor dict (could be int, bool, etc.)
        return item

In [131]:
def clean_n_decode(data_in, file_name):
    splited_data = data_in.replace("b'{", 'b"{').split('b"')
    arr_data = [spld.replace("\\'", "'") for spld in splited_data]

    json_data_arr = []
    
    for i in range(len(arr_data)):
        if i == 0:
            continue
        elif "TikTokApi.video(" in arr_data[i]:
            continue
        else:
            try:
                # clean & decode UTF-8
                content = arr_data[i].replace("'", '"')[:-2]
                content = re.sub(r'([A-Za-z])"(?=[A-Za-z])', r"\1'", content) # reg replace from Don"t to Don't

                # # analyze error
                # pattern1 = r'"title":\s*"(.*?)"'
                # pattern2 = r'"desc":\s*"(.*?)"'
                # updated_string = re.sub(pattern1, '"title": "ignore string error"', content)
                # updated_string = re.sub(pattern2, '"desc": "ignore string error"', updated_string).replace("True", "true").replace("False", "false")

                str_data = ast.literal_eval(content)
                decoded_data = str(decode_utf8(str_data)).replace("'", '"').replace("True", "true").replace("False", "false").replace("\\\\n", " ")
                updated_string = re.sub(r'([A-Za-z])"(?=[A-Za-z])', r"\1'", decoded_data) # reg replace from Don"t to Don't
                # print(updated_string) # print and paste this on the .json file to see if there's an error

                # turn string into object
                json_data = json.loads(updated_string)
                id = json_data["id"]

                # print(id, json_data)

                # # export to check json data
                # file_path = r"./exported-accountpage-json-error-check/" + file_name[:-4] + "-" + id + ".json"
                # with open(file_path, "w", encoding='utf-8') as file:
                #     json.dump(json_data, file, indent=4, ensure_ascii=False)
                
                
                json_data_arr.append(json_data)
                # break # uncomment this for one time loop
            except:
                print(file_name + ' error inside function')
                break
    return json_data_arr


In [132]:
def extract_nested_keys(my_dict):

    keys_to_extract = [
        ["author", "uniqueId"],
        ["author", "signature"],
        ["author", "verified"],
        "desc",
        "verified",
        "createTime",
        "digged",
        "duetDisplay",
        "duetEnabled",
        "forFriend",
        "id",
        "itemCommentStatus",
        ["music", "authorName"],
        ["music", "coverLarge"],
        ["music", "duration"],
        ["music", "id"],
        ["music", "original"],
        ["music", "playUrl"],
        ["music", "title"],
    ]

    result = {}
    for key in keys_to_extract:
        if isinstance(key, list):
            nested_result = my_dict
            for nested_key in key:
                if nested_key in nested_result:
                    nested_result = nested_result[nested_key]
                    # print(nested_key)
                else:
                    nested_result = None
                    break
            if nested_result is not None:
                result[str("_".join(key))] = nested_result
        elif key in my_dict:
            result[key] = my_dict[key]
    return result



In [133]:
def extract_hashtags(contents):
    hashtag_arr = []

    for content in contents:
        for key, textExtra_arr in content.items():
            if key == "textExtra":
                for textExtra in textExtra_arr:
                    for key, hashtagName in textExtra.items():
                        if key == "hashtagName":
                            hashtag_arr.append(hashtagName)

    # print(hashtag_arr)
    joined_string = ', '.join(hashtag_arr)
    return joined_string

In [153]:
def extract_data(json_data):
    try:
        extracted_values = extract_nested_keys(json_data)
        # print(json_data)
        # print(json_data["contents"])
        if json_data["contents"] is None:
            return extracted_values
        else:
            extracted_values["hashtagNames"] = extract_hashtags(json_data["contents"])
            # print(str(extracted_values).replace("'",'"').replace("True", "true").replace("False", "false"))
            # print(type(extracted_values))
            return extracted_values
    except:
        print("while extracting some fileds missing")


In [172]:
def extract_hashtags(contents):
    hashtag_arr = []

    for content in contents:
        for key, textExtra_arr in content.items():
            if key == "textExtra":
                for textExtra in textExtra_arr:
                    for key, hashtagName in textExtra.items():
                        if key == "hashtagName":
                            hashtag_arr.append(hashtagName)

    # print(hashtag_arr)
    # joined_string = ', '.join(hashtag_arr)
    return hashtag_arr

In [135]:
def clear_csv_file(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)

In [178]:
origin_folder_path = r"./txt-accounts-files"
csv_file_path = "AccountPages2.csv"
extracted_df = None

clear_csv_file(csv_file_path)

for i, file_name in enumerate(os.listdir(origin_folder_path)):
    if file_name.endswith('.txt'):
        file_path = os.path.join(origin_folder_path, file_name)
        print("\n", i, file_name)

        # try:
        with open(file_path, "r") as file:
            file_data = file.read()
        
        json_data_arr = clean_n_decode(file_data, file_name)
        extracted_data_json_list = [extract_data(json_data) for json_data in json_data_arr] 
        
        if extracted_df is None:
            extracted_df = pd.DataFrame(columns=extracted_data_json_list[0].keys())

        filtered_list = [data for data in extracted_data_json_list if data is not None] # one or more elements in your extracted_data_json_list is None instead of a dictionary
        df = pd.DataFrame.from_records(filtered_list)
        extracted_df = pd.concat([extracted_df, df], ignore_index=True)

        # break

        
        # for json_data in extracted_data_json_list:
        #     df = pd.DataFrame(json_data)
        #     extracted_df = pd.concat([extracted_df, df], ignore_index=True)
            
        # except:
        #     print(file_name + ' error')

extracted_df.to_csv(csv_file_path, mode='a', header=True, index=False)


 0 1997thantzinoo1.txt
1997thantzinoo1.txt error inside function
while extracting some fileds missing
while extracting some fileds missing

 1 alocatagram.txt

 2 bikesure2.txt
bikesure2.txt error inside function

 3 diary.f_m.txt

 4 evgirl.txt

 5 ffern_123.txt
while extracting some fileds missing
while extracting some fileds missing

 6 flipsmile2.txt
flipsmile2.txt error inside function

 7 geytty.txt
while extracting some fileds missing

 8 jengtieow.txt
jengtieow.txt error inside function

 9 kait000n.txt
kait000n.txt error inside function
while extracting some fileds missing
while extracting some fileds missing

 10 kim_chiangsong.txt

 11 madam.grammy.txt

 12 nammon0012.txt
while extracting some fileds missing
while extracting some fileds missing
while extracting some fileds missing
while extracting some fileds missing
while extracting some fileds missing
while extracting some fileds missing

 13 nastya.jung.txt

 14 pnkkazxz1a.txt

 15 rattanaphon_saechio.txt

 16 umbrellaku

In [128]:
extracted_data_json_list

[]

In [160]:
extracted_df.header()

AttributeError: 'NoneType' object has no attribute 'header'

In [157]:
df

,author_uniqueId,author_signature,author_verified,desc,createTime,digged,duetDisplay,duetEnabled,forFriend,id,itemCommentStatus,music_authorName,music_coverLarge,music_duration,music_id,music_original,music_playUrl,music_title,hashtagNames
0,1997thantzinoo1,FB = Thant Zin Oo page = 1997kothantzin Line =...,False,@Win Thu Aung,1697677280,False,0,True,False,7291468383554407685,0,🖤MyoThiHaKyaw🖤,https://p16-sign-sg.tiktokcdn.com/aweme/1080x1...,23,7260431444898581250,False,https://v16-webapp-prime.tiktok.com/video/tos/...,original sound - 🖤MyoThiHaKyaw🖤,


In [79]:
origin_folder_path = r"./txt-accounts-files"
csv_file_path = "AccountPages.csv"
extracted_df = None

clear_csv_file(csv_file_path)

dataframes = []

for i, file_name in enumerate(os.listdir(origin_folder_path)):
    if file_name.endswith('.txt'):
        file_path = os.path.join(origin_folder_path, file_name)

        try:
            with open(file_path, "r") as file:
                file_data = file.read()

            json_data_arr = clean_n_decode(file_data, file_name)
            extracted_data_json_list = [extract_data(json_data) for json_data in json_data_arr]

            if i == 0:
                extracted_df = pd.DataFrame(columns=extracted_data_json_list[0].keys())

            df = pd.DataFrame(extracted_data_json_list)
            dataframes.append(df)

        except:
            print(file_name + ' error')

extracted_df = pd.concat(dataframes, ignore_index=True)
extracted_df.to_csv(csv_file_path, mode='a', header=True, index=False, encoding='utf-8')

1997thantzinoo1.txt error inside function
1997thantzinoo1.txt error
bikesure2.txt error inside function
ffern_123.txt error
flipsmile2.txt error inside function
geytty.txt error
jengtieow.txt error inside function
kait000n.txt error inside function
kait000n.txt error
nammon0012.txt error
yfamilyyy.txt error inside function
Yok_jittraphan.txt error
